# Машинное обучение

## Факультет математики НИУ ВШЭ

### 2019-2020 учебный год

Илья Щуров, Соня Дымченко, Руслан Хайдуров, Александр Каган, Павел Балтабаев

# Семинар 15 (16)
---

# K-means

<img src="http://dendroid.sk/wp-content/uploads/2013/01/kmeansimg-scaled1000.jpg" width=400>

Алгоритм k-means применяется для задачи кластеризации. Напомню его основные шаги. Пусть у нас имеется $N$ точек на плоскости ${(x_1,y_1),...,(x_N,y_N)}$
Допустим мы хотим разбить их на $k=2$ кластера.
1. Выбираем $k=2$ случайные точки из этого множества. Говорим, что они являются теперь центрами наших кластеров.
2. Для каждой из оставшихся точек смотрим, к какому из центров она ближе и определяем её в этот кластер.
3. У нас получилось разбить точки на 3 кластера. Естественно это не оптимальное разбиение. Найдём новые центры кластеров. Например, если точки ${(x_{i_1}, y_{i_1}),...,(x_{i_n}, y_{i_n})}$ попали в один кластер, то их новый центр будет имеет координаты:
$$x_M=\frac{x_{i_1}+...+x_{i_n}}{n}$$
$$y_M=\frac{y_{i_1}+...+y_{i_n}}{n}$$
4. Переходим к шагу 2 и продолжаем до тех пор, пока кластеры не перестанут меняться.

[Ссылка на визуализацию](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_circles

np.random.seed(42)

plt.figure(figsize=(16, 5))

X_noised, y_noised = make_blobs(n_samples=1500, centers=3, cluster_std=2.1, random_state=42)
plt.subplot(131)
plt.scatter(X_noised[:, 0], X_noised[:, 1], edgecolors='face')
plt.axis('equal') # чтобы рисунок был не растянут
plt.title(u"Круглые кластеры", fontsize=15)

transformation = [[0.60834549, -0.63667341], [-0.40887718, 0.85253229]] # матрица преобразования
X_long = np.dot(X_noised, transformation) # умножим на нее, чтобы растянуть кучки
y_long = y_noised

plt.subplot(132)
plt.scatter(X_long[:, 0], X_long[:, 1], edgecolors='face')
plt.title(u"Вытянутые кластеры", fontsize=15)

X_circles, y_circles = make_circles(n_samples=1500, factor=0.5, noise=0.05)
plt.subplot(133)
plt.scatter(X_circles[:, 0], X_circles[:, 1], edgecolors='face')
plt.title(u"Кластеры сложной структуры", fontsize=15)

plt.show()

In [ ]:
logging = dict(blob=[], long=[], circle=[])

### K-means из sklearn

Воспользуйтесь реализацией алгоритма из библиотеки sklearn и провизуализируйте результат кластеризации на $k=2,3,5,7$ классов для трех датасетов выше, сохраните результат, провизуализируйте (получится сетка графиков 4 на 3)

In [ ]:
from sklearn.cluster import KMeans

# твой код тут # (つ▀¯▀)つ

# Спектральная кластеризация

Состоит из 3х шагов:

1. Для входных данных размерностью $(N, d)$ считаем матрицу схожести $A$ размерностью $(N,N)$. В простейшем случае это могут быть попарные расстояния, но на самом деле матрица может быть получена и с использованием более сложных операций над парами точек 
2. Полученная матрица симметрична. Вычисляем для нее матрицу Лапласиана:
    $${\displaystyle L=D-A}$$
    где ${\displaystyle D}$  диагональная матрица, такая что:
    $${\displaystyle D_{ii}=\sum _{j}A_{ij}.}$$
    В самом алгоритме используется нормализованная версия Лапласиана
3. Подсчет собственных векторов для матрицы Лапласиана и кластеризация этих векторов подручными методами, например, с помощью того же K-means

In [ ]:
import scipy as sp
from sklearn.metrics import pairwise_distances


def laplacian(A):
    """Computes the symetric normalized laplacian.
    L = D^{-1/2} A D^{-1/2}
    """
    D = np.zeros(A.shape)
    w = np.sum(A, axis=0)
    D.flat[::len(w) + 1] = w ** (-0.5)  # set the diag of D to w
    return D.dot(A).dot(D)


def k_means(X, n_clusters):
    kmeans = KMeans(n_clusters=n_clusters, random_state=1231)
    return kmeans.fit(X).labels_


def spectral_clustering(affinity, n_clusters, cluster_method=k_means):
    L = laplacian(affinity)
    eig_val, eig_vect = sp.sparse.linalg.eigs(L, n_clusters)
    X = eig_vect.real
    rows_norm = np.linalg.norm(X, axis=1, ord=2)
    Y = (X.T / rows_norm).T
    labels = cluster_method(Y, n_clusters)
    return labels

In [ ]:
A = pairwise_distances(X_noised)
A.shape

In [ ]:
labels = spectral_clustering(A, n_clusters=3)
points_x = X_noised[:,0]
points_y = X_noised[:,1]
plt.title('Спектральная кластеризация')
plt.scatter(x=points_x, y=points_y, c=labels)
plt.show()

### Спектральная кластеризация из sklearn

Сделайте кластеризацию с помощью sklearn для тех же трех датасетов с числом кластеров $k=3,4,5,7$, сохраните результат, провизуализируйте.

In [ ]:
from sklearn.cluster import spectral_clustering

# твой код тут # (つ▀¯▀)つ

# Агломеративная кластеризация

Наверное самый простой и понятный алгоритм кластеризации без фиксированного числа кластеров — агломеративная кластеризация.

Интуиция у алгоритма очень простая:

1. Начинаем с того, что высыпаем на каждую точку свой кластер
2. Сортируем попарные расстояния между центрами кластеров по возрастанию
3. Берём пару ближайших кластеров, склеиваем их в один и пересчитываем центр кластера
4. Повторяем п. 2 и 3 до тех пор, пока все данные не склеятся в один кластер

Сам процесс поиска ближайших кластеров может происходить с использованием разных методов объединения точек:

* Single linkage — минимум попарных расстояний между точками из двух кластеров
* Complete linkage — максимум попарных расстояний между точками из двух кластеров
* Average linkage — среднее попарных расстояний между точками из двух кластеров
* Centroid linkage — расстояние между центроидами двух кластеров


In [ ]:
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist

X = np.zeros((150, 2))

np.random.seed(seed=42)
X[:50, 0] = np.random.normal(loc=0.0, scale=.3, size=50)
X[:50, 1] = np.random.normal(loc=0.0, scale=.3, size=50)

X[50:100, 0] = np.random.normal(loc=2.0, scale=.5, size=50)
X[50:100, 1] = np.random.normal(loc=-1.0, scale=.2, size=50)

X[100:150, 0] = np.random.normal(loc=-1.0, scale=.2, size=50)
X[100:150, 1] = np.random.normal(loc=2.0, scale=.5, size=50)

distance_mat = pdist(X) # pdist посчитает нам верхний треугольник матрицы попарных расстояний

Z = hierarchy.linkage(distance_mat, 'single') # linkage — реализация агломеративного алгоритма
plt.figure(figsize=(18, 5))
dn = hierarchy.dendrogram(Z, color_threshold=0.5)

### Аггломеративная кластеризация из sklearn
С помощью sklearn сделаем аггломеративную кластеризацию с числом кластеров $k=3,10$ для разных параметров.

In [ ]:
import time

from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

data = X_noised # X_circles, X_long

knn_graph = kneighbors_graph(data, 10, include_self=False)

for connectivity in (None, knn_graph):
    for n_clusters in (3, 10):
        plt.figure(figsize=(10, 4))
        for index, linkage in enumerate(('average',
                                         'complete',
                                         'ward')):
            plt.subplot(1, 3, index + 1)
            model = AgglomerativeClustering(linkage=linkage,
                                            connectivity=connectivity,
                                            n_clusters=n_clusters)
            t0 = time.time()
            model.fit(data)
            elapsed_time = time.time() - t0
            plt.scatter(data[:, 0], data[:, 1], c=model.labels_,
                        cmap=plt.cm.nipy_spectral)
            plt.title('linkage=%s\n(time %.2fs)' % (linkage, elapsed_time),
                      fontdict=dict(verticalalignment='top'))
            plt.axis('equal')
            plt.axis('off')

            plt.subplots_adjust(bottom=0, top=0.8, wspace=0,
                                left=0, right=1)
            plt.suptitle('n_cluster=%i, connectivity=%r' %
                         (n_clusters, connectivity is not None), size=17)


plt.show()

# Метрики кластеризации

### Adjusted Rand Index (ARI)

Как можно догадаться по названию, это "подкрученная" метрика `Random Index`:

${\displaystyle RI(S)={\frac {a+b}{a+b+c+d}}={\frac {a+b}{n \choose 2}}}$

Для набора $S$ из $n$ элементов и его двух возможных разбиений на кластера  $X$ и $Y$

* $  a$, количество пар элементов в $  S$, которые находятся в одном и том же подмножестве в  $X$ и в одном и том же подмножестве в $  Y$
* $b$, количество пар элементов в $S$, которые находятся в разных подмножествах в $ X$ и в разных подмножествах в $  Y$

* $  c$, количество пар элементов в $ S$, которые находятся в одном и том же подмножестве в $  X$ и в разных подмножествах в $  Y$
* $  d$, количество пар элементов в $ S$, которые находятся в разных подмножествах в $  X$ и в одном и том же подмножестве в $  Y$

 Метрика $ARI$ делает поправку на то, чтобы случайное разметка $S$ на кластера имела бы метрику равной нулю.

$${\displaystyle \overbrace {ARI} ^{\text{Adjusted Index}}={\frac {\overbrace {\sum _{ij}{\binom {n_{ij}}{2}}} ^{\text{Index}}-\overbrace {[\sum _{i}{\binom {a_{i}}{2}}\sum _{j}{\binom {b_{j}}{2}}]/{\binom {n}{2}}} ^{\text{Expected Index}}}{\underbrace {{\frac {1}{2}}[\sum _{i}{\binom {a_{i}}{2}}+\sum _{j}{\binom {b_{j}}{2}}]} _{\text{Max Index}}-\underbrace {[\sum _{i}{\binom {a_{i}}{2}}\sum _{j}{\binom {b_{j}}{2}}]/{\binom {n}{2}}} _{\text{Expected Index}}}}}$$

In [ ]:
from sklearn.metrics import adjusted_rand_score


# Silhouette

Если вдруг все же надо подобрать значение k достаточно хорошим, можно смотреть на визуализацию метрики силуэт.

Для ее подсчета не требуется знать истинную разметку, она лишь характеризует текущую.

Wiki:
>Let ${\displaystyle b(i)} $ be the smallest average distance of ${\displaystyle i}$  to all points in any other cluster, of which ${\displaystyle i}$  is not a member. The cluster with this smallest average dissimilarity is said to be the "neighbouring cluster" of ${\displaystyle i}$  because it is the next best fit cluster for point ${\displaystyle i}$. We now define a silhouette:

$${\displaystyle s(i)={\frac {b(i)-a(i)}{\max\{a(i),b(i)\}}}}$$

Таким образом, чем выше ситуэт для объекта, тем глубже этот объект находится в своем кластере.

Это также может помочь детектировать выбросы в данных.

Пример визуализации:
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Silhouette-plot-orange.png/800px-Silhouette-plot-orange.png" width=400>

[Пример подбора параметра K для Kmeans с помощью `Silhouette`](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html)

In [ ]:
from sklearn.metrics import silhouette_score

### Посчитаем метрики
Посчитайте метрику ARI для всех использованных вами алгоритмов кластеризации для датасетов.

In [ ]:
# твой код тут # (つ▀¯▀)つ

# Кластеризация картинки

Найдите в интернете картинку, подгрузите ее и примените алгоритм кластеризации. 

Посмотрите что вышло.

In [ ]:
from PIL import Image
import urllib

def load_image(infilename):
    if infilename.startswith('https'):
        f = urllib.request.urlopen(infilename)
        img = Image.open(f)
    else:
        img = Image.open(infilename)
    img.load()
    data = np.asarray(img, dtype="float32")
    return data

def save_image(img,outfilename):
    img.save(outfilename)
    

url = # твоя ссылка тут # (つ▀¯▀)つ
pic = load_image(url)
print("image shape: ", pic.shape)
plt.imshow(pic[:,:,1])
plt.show()

Для картинки нужно уменьшить число цветов в палитре; для этого нужно выделить кластеры в пространстве RGB, объекты соответствуют пикселям изображения; после выделения кластеров, все пиксели, отнесенные в один кластер, заполняются одним цветом; этот цвет может быть центроидом соответствующего кластера, медианным цветом по кластеру. Попробуйте различные алгоритмы кластеризации:

* KMeans
* DBSCAN
Можно использовать и другие:

* MeanShift
* AgglomerativeClustering

Какие угодно, какие сможете найти
Рассмотрите число кластеров K = 2, 3, 10, 20 (в алгоритмах, где есть такой гиперпараметр).

Для различных кластеризаций оцените и сравните потери от уменьшения цветов при помощи метрики [SSIM](http://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.compare_ssim). Какой способ оказался лучшим?